In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()

In [ ]:
df = pd.DataFrame(housing['data'], columns=housing['feature_names'])
df['target'] = housing['target']
#The target variable is the median house value for California districts,expressed in hundreds of thousands of dollars ($100,000).

In [ ]:
df.shape

(20640, 9)

In [ ]:
df

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
...,...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847


In [ ]:
def data_transform(df):

  # Splitting data into training and testing data
  df = df.sample(frac=1).reset_index(drop=True)
  no_of_train_rows = int(np.floor(0.80*df.shape[0]))
  train_df = df.iloc[0:no_of_train_rows,:]
  train_df = df.sort_values("target")
  test_df = df.iloc[no_of_train_rows:,:]

  l = df.shape[1]
  train_x = train_df.iloc[:,0:l-2]
  train_y = train_df.iloc[:,-1]
  test_x = test_df.iloc[:,0:l-2]
  test_y = test_df.iloc[:,-1]
  train_x = np.array(train_x.iloc[0:5000,:])
  train_y = np.array(train_y.iloc[0:5000])


  return train_x,train_y,test_x,test_y

In [ ]:
train_x,train_y,test_x,test_y = data_transform(df)

In [ ]:
train_x[0:500]

array([[   2.1       ,   19.        ,    3.77439024, ...,  490.        ,
           2.98780488,   36.4       ],
       [   4.1932    ,   52.        ,    3.56888889, ...,  628.        ,
           2.79111111,   34.24      ],
       [   1.6607    ,   16.        ,    6.71052632, ...,   85.        ,
           2.23684211,   39.71      ],
       ...,
       [   2.0577    ,   12.        ,    6.37272727, ...,  434.        ,
           3.94545455,   33.79      ],
       [   1.516     ,   28.        ,    4.40327869, ..., 1331.        ,
           4.36393443,   36.59      ],
       [   2.101     ,   51.        ,    5.31111111, ...,  662.        ,
           3.67777778,   36.73      ]])

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#################################################################
# User defined storage class :To store information as we process 
# in building our XGBoostTree
# 1. Stores the data corresponding to the particular node class
# 2. Stores the feature index giving the best split for that iteration
# 3. Stores the threshold or can also be called the BestSplitVal 
#    corresponding to the index
# 4. Node_val corresponds to the leaf value. Defaulted to 10000
# 5. Node.left & Node.right corresponds to the location of 
#    future nodes down the XGBoost Tree Structure, can be 
#    thought of linkedList from basic data structures.
#################################################################
class Node:

  def __init__(self):
    self.data = None
    self.feature_index = None
    self.threshold = None 
    self.node_val = 10000 #Default Value
    self.left = None
    self.right = None


In [ ]:
#######################################################################
# Similarity score is calcuated to understand the purtiy of the node.
# Higher the similarity score, purer is the done. 
#######################################################################

def similarityScore(x,y,residuals,lambda1):

  similarityScore = (np.sum(residuals))**2/(len(y)+lambda1)

  #Dividing the data into left and right branch by taking the average of all the
  # adjacent training data

  #######################################################################
  # Initializing all the lists and variables needed in executing the 
  # function
  #######################################################################

  gainLst = []
  splitVallst = []
  gainThreshold = 0
  SplitValThreshold = 0
  Index = 0
  featureIndex = 0

  #######################################################################
  # Outer forLoop corresponding to the number of features in the dataset.
  #######################################################################

  for j in range(0,x.shape[1]):
    gainThreshold = 0
    #######################################################################
    # Inner forLoop across all data points, calculating the average of 
    # adjacent data points, and calculating the best fit
    # In case of XGBoost, best fit is the one, giving the maximum gain.
    #######################################################################
    for i in range(1,x.shape[0]-1):
      
      splitVal = (x[i-1,j] + x[i,j])/2

      # create two halfs based on the split val 

      left_df = x[x[:,j]<= splitVal]
      right_df = x[x[:,j] > splitVal]

      left_residuals = residuals[x[:,j]<= splitVal]
      right_residuals = residuals[x[:,j] > splitVal]

      # now calculate the Similarity Score for both Halves

      similarityLeftScore = (np.sum(left_residuals))**2/(len(left_residuals) + lambda1)
      similarityRightScore = (np.sum(right_residuals))**2/(len(right_residuals) + lambda1)

      # Now we Calculate the Branch Gain 
      #######################################################################
      # Inner forLoop across all data points, calculating the average of 
      # adjacent data points, and calculating the best fit
      # In case of XGBoost, best fit is the one, giving the maximum gain.
      #######################################################################
      gain = similarityLeftScore + similarityRightScore - similarityScore

      if gain > gainThreshold:
        gainThreshold = gain
        SplitValThreshold = splitVal
        Index = i

    gainLst.append(gainThreshold)
    splitVallst.append(SplitValThreshold)

  return gainLst,splitVallst

In [ ]:
# Recursive Building the XGBoostTree

def buildTree(x,y,node,residuals,maxDepth,lambda1):

  gainlst,splitvalLst = similarityScore(x,y,residuals,lambda1)

  index = np.argmax(gainlst)
  splitVal = splitvalLst[index]
  gain = gainlst[index]

  # Base Case 
  if len(residuals) == 1:
     node.node_val = residuals/(len(residuals)+lambda1)
     return node

  if gain <0 or maxDepth == 0:
    # Calcate the node value of the tree and return
    node.node_val = np.sum(residuals)/(len(residuals) + lambda1)
    return node


  node.feature_index = index
  node.threshold = splitVal
  
  # Splitting the data

  train_left_x = x[x[:,index]<= splitVal]
  train_right_x = x[x[:,index] > splitVal]

  
  train_left_y = y[x[:,index]<= splitVal]
  train_right_y = y[x[:,index] > splitVal]
  
  residual_left = residuals[x[:,index]<= splitVal]
  residual_right = residuals[x[:,index] > splitVal]

  leftNode = Node()
  rightNode = Node()

  maxDepth = maxDepth - 1

  node.left = buildTree(train_left_x,train_left_y,leftNode,residual_left,maxDepth,lambda1)
  node.right = buildTree(train_right_x,train_right_y,rightNode,residual_right,maxDepth,lambda1)

  return node

In [ ]:
##########################################################
#Predict 
#Input : The test dataset, root of the tree, list of columns
#Output : Returns the predicted value.
##########################################################
def predict(x,treenode):

  #Base Case
  if treenode.node_val != 10000:
    return treenode.node_val

  head = treenode
  index = head.feature_index
  threshold = head.threshold

  value = x[index]
 
  if value <= threshold:
     pred = predict(x,head.left)
  else:
    pred = predict(x,head.right)
  
  return pred  

def predictNewValues(x,treenode):
  Leafresiduals = []
  for i in range(0,len(x)):
    arrx = x[i,:]
    res = predict(arrx,treeNode)
    Leafresiduals.append(res)

  return Leafresiduals


In [ ]:
#######################################################################
# XGBoost Algorithm main function.
#######################################################################
def XGBoost(x,y,maxDepth,NoFTrees,eta):

  trees = []
  node = Node()
  # Initial Prediction
  #######################################################################
  # Very Initial Prediction at level-0
  # It is mostly mean of the label training dataset.
  #######################################################################
  pred = np.mean(y)

  # Calculate the residuals
  residuals = y - pred
  newPred=np.repeat(pred,len(y))
  
  for k in range(NoFTrees):
    print("Tree Number",k)
    node = Node()
    #######################################################################
    # building XGBoost tree, trying to predict the residuals using the 
    # feature of the training dataset.
    #######################################################################
    treeNode = buildTree(x,y,node,residuals,maxDepth,lambda1=1)
    trees.append(treeNode)

    # get new residuals
    res = predictNewValues(x,trees[k])
    res = np.array(res).ravel()
    g = eta * res

    #######################################################################
    # Calculating new prediction
    # Newpred = Previous predictions + learningrate * residuals
    #######################################################################
  
    newPred  = newPred + g

    residuals = y - newPred

  return pred, trees

In [ ]:
pred, Alltrees = XGBoost(train_x,train_y,6,20,0.3)

Tree Number 0
Tree Number 1
Tree Number 2
Tree Number 3
Tree Number 4
Tree Number 5
Tree Number 6
Tree Number 7
Tree Number 8
Tree Number 9
Tree Number 10
Tree Number 11
Tree Number 12
Tree Number 13
Tree Number 14
Tree Number 15
Tree Number 16
Tree Number 17
Tree Number 18
Tree Number 19


In [ ]:
### Prediction 
x = np.array(test_x.iloc[0:500,:])
FirstPred=np.repeat(pred,len(x))
prediction = FirstPred
# get new residuals
for i in range(0,len(Alltrees)):
  res = predictNewValues(x,Alltrees[i])
  res = np.array(res).ravel()
  prediction = prediction + 0.3 * res

In [ ]:
# Accuracy
MeanSquareError = np.sqrt(1/len(test_y[0:500])*np.sum((test_y[0:500] - prediction)**2))

print(MeanSquareError)


1.575443150283064


# SKLEARN Library

In [ ]:
def data_transform(df):

  # Splitting data into training and testing data
  df = df.sample(frac=1).reset_index(drop=True)
  no_of_train_rows = int(np.floor(0.80*df.shape[0]))
  train_df = df.iloc[0:no_of_train_rows,:]
  train_df = df.sort_values("target")
  test_df = df.iloc[no_of_train_rows:,:]

  l = df.shape[1]
  train_x = train_df.iloc[:,0:l-2]
  train_y = train_df.iloc[:,-1]
  test_x = test_df.iloc[:,0:l-2]
  test_y = test_df.iloc[:,-1]



  return train_x,train_y,test_x,test_y

In [ ]:
train_x,train_y,test_x,test_y = data_transform(df)

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

#data_dmatrix = xgb.DMatrix(data=train_x,label=y)
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

xg_reg.fit(train_x,train_y)

preds = xg_reg.predict(test_x)

rmse = np.sqrt(mean_squared_error(test_y, preds))
print("RMSE: %f" % (rmse))

RMSE: 1.038412
